# Laboratorium 6

In [1]:
import math
from objsize import get_deep_size

Zaimportujmy przydatne biblioteki.

Najpierw zaimportujmy dane, które będziemy wykorzystywać

In [2]:
def readfile(name):
    data = ""
    with open(name, 'r') as file:
        data = file.read()
    return data

In [3]:
rij = readfile("romeo-i-julia-700.txt")
zad = readfile("zad6")
data = readfile("1997_714.txt")

In [4]:
!cat zad6 romeo-i-julia-700.txt 1997_714.txt | grep '~'
!cat zad6 romeo-i-julia-700.txt 1997_714.txt | grep '\$'

In [5]:
ord('~')>ord('z')>ord('Z')

True

Jak widać znak '~' nie występuje w danych wejściowych i jest najpóźniej leksykograficznie. Również '\\$' nie występuje w danych wejściowych i przyda się do połączenia pat\\$text, ponieważ '&' nie jest unikalny.

### 1. Zaimplementować budowę słownika podstawowych składowych.

Najpierw zaimplementujmy funkcję porządkującą etykietowanie ciągu sekwencji.

In [6]:
def sortNRename(seq):
    last = None
    ix = 0
    posix = [None for i in range(len(seq))]
    first = {}
    for el in sorted([(e, i) for i,e in enumerate(seq)]):
        if last and last[0] != el[0]:
            ix += 1
            first[ix] = el[1]
        posix[el[1]] = ix
        if last is None:
            first[0] = el[1]
        last = el
    return (posix, first)

In [7]:
sortNRename([(1,2), (3,1), (2,2), (1,1), (2,3), (1,2)])

([1, 4, 2, 0, 3, 1], {0: 3, 1: 0, 2: 2, 3: 4, 4: 1})

Rozwiązanie zgodne z oczekiwaniami.

Teraz zaimplementujmy algorytm KMR, który zbuduje nam BDF.

In [8]:
def kmr(txt):
    length = len(txt)
    fac = math.floor(math.log2(len(txt)))
    maxlen = 2**fac
    pad = 2**(fac+1)-1
    txt += '~'*pad
    posix, first = sortNRename(list(txt))
    names = {1: posix}
    els = {1: first}
    for i  in range(1, fac):
        p = 2**(i-1)
        new = []
        for j in range(len(txt)):
            if j+p < len(names[p]):
                new += [(names[p][j], names[p][j+p])]
        posix, first = sortNRename(new)
        names[p*2] = posix
        els[p*2] = first
    return (names, els)

In [9]:
text = "abbabbaba"
names, entries = kmr(text)
print("names:")
for k,v in names.items():
    print(k, [e+1 for e in v[:len(text)]])
print("\npositions:")
for k,v in entries.items():
    print(k, [v[e]+1 for e in range(len(v)-1)])

names:
1 [1, 2, 2, 1, 2, 2, 1, 2, 1]
2 [1, 4, 3, 1, 4, 3, 1, 3, 2]
4 [2, 7, 5, 2, 7, 4, 1, 6, 3]

positions:
1 [1, 2]
2 [1, 9, 3, 2]
4 [7, 1, 9, 6, 3, 8, 2]


Rozwiązanie zgodne z oczekiawniami.

### 2. Zaimplementować wyszukiwanie wzorca korzystając z DBF. Wyszukiwanie ma działać dla każdej długości wzorca nie większej od długości całego tekstu. Czy konieczne jest budowanie nowego DBF dla pat&text?

Najpierw zróbmy wersję domyślną z budowaniem nowego DBF dla pat&text.

In [10]:
def findPatTxt(pat, txt, pri=True):
    assert len(pat)<len(txt)
    names, posix = kmr(pat+'$'+txt)
    res=[]
    if len(pat) in names:
        nm=names[len(pat)]
        for i in range(len(pat)+1, len(nm)):
            if nm[i]==nm[0]:
                res += [i-len(pat)-1]
    else:
        fac = math.floor(math.log2(len(pat)))
        pos = len(pat) - 2**fac
        nm = names[2**fac]
        for i in range(len(pat)+1, len(nm)-(len(pat)-fac)):
            if nm[i]==nm[0] and nm[i+pos]==nm[pos]:
                res +=[i-len(pat)-1]
        
    if pri:
        print(f"Found: {len(res)}")
        print(res)
    return res, len(res)    

In [11]:
findPatTxt("aab", "babaaab");

Found: 1
[4]


In [12]:
findPatTxt("AAAA","AAAAABAAABA")
findPatTxt("AAAAB","AAAAAAAAAAAAAAAAAB")
findPatTxt("ABABAC","ABABABCABABABCABABABC");

Found: 2
[0, 1]
Found: 1
[13]
Found: 0
[]


Jak widać wyniki zgodne z oczekiawnymi.

Teraz zastanówmy się, czy musimy za każdym razem budować DBF.\
Dotychczasowo tworzyliśmy pattern\\$text, żeby wliczyć pattern do budowy DBF, co znacząco ułatwia sprawdzanie czy podany pattern występuje w tekście.\
Wykorzystujemy to w ten sposób, że możemy z łatwością sprawdzić, jakie "nazwy" otrzymały odpowiednie indeksy patternu i później przeszukując text, możemy szukać tylko tych indeksów na odpowiednio odległych pozycjach.\
Jendak, dzięki wykorzystaniu tablicy positions/elements/entires (druga z krotki kmr), jesteśmy w stanie łatwo odczytać jaki ciąg jak jest "nazwany".\
Co więcej, podane etykiety są już posortowane leksykograficznie (po fragmencie który reprezentują), co umożliwia nam binsearchowanie odpowiednich etykiet dla naszego patternu.\
Tym sposobem możemy budować DBF tylko jednokrotnie dla konkretnego tekstu, niezależnie od liczby różnych szukanych wzorców, musimy jednak każdorazowo zidentyfikować odpowiednie etykiety dla nowego patternu, co możemy zrobić wykorzystując binsearch - zawsze max 2 binsearch'e (na prefix i sufix), które w sumie pokrywają cały text i każdy jest długości największej mniejszej potęgi dwójki (niż długość patternu).

Otrzymujemy w ten sposób złożoności:\
O(nlogn) - pamięciowo - dla n = długość textu - niezależnie od patternu.\
O(nlogn) - czasowo - dla n = długość textu - niezależnie od patternu - tworzenie DBF.\
O(nlogk) - czasowo - dla n = długość patternu i k = liczba unikalnych podsłów textu, długości n (a bardziej największej potęgi 2, będącej mniejszą od n).\
O(n) - czasowo - dla n = długość textu - niezależnie od patternu - znalezienie wszystkich wystąpień patternu w tekście, mając już etykiety.

In [13]:
def binsearch(left, right, test):
    if left>right:
        return -1
    else:
        k = (left+right)//2
        res = test(k)
        if res==0:
            return k
        elif res>0:
            return binsearch(left, k-1, test)
        else:
            return binsearch(k+1, right, test)

In [14]:
print(binsearch(1, 100, lambda x: x-100))
binsearch(1, 100, lambda x: x-13)

100


13

In [15]:
"as">"bs"

False

In [16]:
def find(pat, txt, wej=None, pri=True):
#     fac = math.floor(math.log2(len(txt)))
#     maxlen = 2**fac
#     pad = 2**(fac+1)-1
#     txt += '~'*pad
    fac = math.floor(math.log2(len(pat)))
    maxlen = 2**fac
    dif = len(pat)-maxlen
    if wej==None:
        names, posix = kmr(txt)
    else:
        names, posix = wej
    posix = posix[maxlen]
#     print(posix)
    name = names[maxlen]
    pref = pat[:maxlen]
    suf = pat[-maxlen:]
#     print(pat, len(pat), 2**fac)
#     print(pref, suf)
    def test1(k):
        if txt[posix[k]:posix[k]+len(pref)] == pref:
            return 0
        elif txt[posix[k]:posix[k]+len(pref)]>pref:
            return 1
        else:
            return -1
    def test2(k):
        if txt[posix[k]:posix[k]+len(suf)] == suf:
            return 0
        elif txt[posix[k]:posix[k]+len(suf)] > suf:
            return 1
        else:
            return -1
#     print(len(posix))
    pix = binsearch(0, len(posix)-1, test1)
    six = binsearch(0, len(posix)-1, test2)
#     print(pix, six)
    res = []
    for i in range(len(txt)-maxlen):
        if name[i]==pix and name[i+dif]==six:
            res += [i]
    if pri:
        print(f"Found: {len(res)}")
        print(res)
    return res, len(res)

In [17]:
find("aab", "babaaab")
find("AAAA","AAAAABAAABA")
find("AAAAB","AAAAAAAAAAAAAAAAAB")
find("ABABAC","ABABABCABABABCABABABC");

Found: 1
[4]
Found: 2
[0, 1]
Found: 1
[13]
Found: 0
[]


Jak widać funkcja działa zgodnie z oczekiwaniami.

### 3. Sprawdzić rzeczywisty czas budowy DBF dla załączonych plików (3 pliki). Porównać z czasem budowy drzewa sufiksów.

Wykorzystamy implementację drzewa sufiksów:

In [18]:
class Node(object):
    """A node in the suffix tree. 
    
    suffix_node
        the index of a node with a matching suffix, representing a suffix link.
        -1 indicates this node has no suffix link.
    """
    def __init__(self):
        self.suffix_node = -1   

    def __repr__(self):
        return "Node(suffix link: %d)"%self.suffix_node

class Edge(object):
    """An edge in the suffix tree.
    
    first_char_index
        index of start of string part represented by this edge
        
    last_char_index
        index of end of string part represented by this edge
        
    source_node_index
        index of source node of edge
    
    dest_node_index
        index of destination node of edge
    """
    def __init__(self, first_char_index, last_char_index, source_node_index, dest_node_index):
        self.first_char_index = first_char_index
        self.last_char_index = last_char_index
        self.source_node_index = source_node_index
        self.dest_node_index = dest_node_index
        
    @property
    def length(self):
        return self.last_char_index - self.first_char_index

    def __repr__(self):
        return 'Edge(%d, %d, %d, %d)'% (self.source_node_index, self.dest_node_index 
                                        ,self.first_char_index, self.last_char_index )


class Suffix(object):
    """Represents a suffix from first_char_index to last_char_index.
    
    source_node_index
        index of node where this suffix starts
    
    first_char_index
        index of start of suffix in string
        
    last_char_index
        index of end of suffix in string
    """
    def __init__(self, source_node_index, first_char_index, last_char_index):
        self.source_node_index = source_node_index
        self.first_char_index = first_char_index
        self.last_char_index = last_char_index
        
    @property
    def length(self):
        return self.last_char_index - self.first_char_index
                
    def explicit(self):
        """A suffix is explicit if it ends on a node. first_char_index
        is set greater than last_char_index to indicate this.
        """
        return self.first_char_index > self.last_char_index
    
    def implicit(self):
        return self.last_char_index >= self.first_char_index

        
class SuffixTree(object):
    """A suffix tree for string matching. Uses Ukkonen's algorithm
    for construction.
    """
    def __init__(self, string, case_insensitive=False):
        """
        string
            the string for which to construct a suffix tree
        """
        self.string = string
        self.case_insensitive = case_insensitive
        self.N = len(string) - 1
        self.nodes = [Node()]
        self.edges = {}
        self.active = Suffix(0, 0, -1)
        if self.case_insensitive:
            self.string = self.string.lower()
        for i in range(len(string)):
            self._add_prefix(i)
    
    def __repr__(self):
        """ 
        Lists edges in the suffix tree
        """
        curr_index = self.N
        s = "\tStart \tEnd \tSuf \tFirst \tLast \tString\n"
        values = list(self.edges.values())
        values.sort(key=lambda x: x.source_node_index)
        for edge in values:
            if edge.source_node_index == -1:
                continue
            s += "\t%s \t%s \t%s \t%s \t%s \t"%(edge.source_node_index
                    ,edge.dest_node_index 
                    ,self.nodes[edge.dest_node_index].suffix_node 
                    ,edge.first_char_index
                    ,edge.last_char_index)
                    
            
            top = min(curr_index, edge.last_char_index)
            s += self.string[edge.first_char_index:top+1] + "\n"
        return s
            
    def _add_prefix(self, last_char_index):
        """The core construction method.
        """
        last_parent_node = -1
        while True:
            parent_node = self.active.source_node_index
            if self.active.explicit():
                if (self.active.source_node_index, self.string[last_char_index]) in self.edges:
                    # prefix is already in tree
                    break
            else:
                e = self.edges[self.active.source_node_index, self.string[self.active.first_char_index]]
                if self.string[e.first_char_index + self.active.length + 1] == self.string[last_char_index]:
                    # prefix is already in tree
                    break
                parent_node = self._split_edge(e, self.active)
        

            self.nodes.append(Node())
            e = Edge(last_char_index, self.N, parent_node, len(self.nodes) - 1)
            self._insert_edge(e)
            
            if last_parent_node > 0:
                self.nodes[last_parent_node].suffix_node = parent_node
            last_parent_node = parent_node
            
            if self.active.source_node_index == 0:
                self.active.first_char_index += 1
            else:
                self.active.source_node_index = self.nodes[self.active.source_node_index].suffix_node
            self._canonize_suffix(self.active)
        if last_parent_node > 0:
            self.nodes[last_parent_node].suffix_node = parent_node
        self.active.last_char_index += 1
        self._canonize_suffix(self.active)
        
    def _insert_edge(self, edge):
        self.edges[(edge.source_node_index, self.string[edge.first_char_index])] = edge
        
    def _remove_edge(self, edge):
        self.edges.pop((edge.source_node_index, self.string[edge.first_char_index]))
        
    def _split_edge(self, edge, suffix):
        self.nodes.append(Node())
        e = Edge(edge.first_char_index, edge.first_char_index + suffix.length, suffix.source_node_index, len(self.nodes) - 1)
        self._remove_edge(edge)
        self._insert_edge(e)
        self.nodes[e.dest_node_index].suffix_node = suffix.source_node_index  ### need to add node for each edge
        edge.first_char_index += suffix.length + 1
        edge.source_node_index = e.dest_node_index
        self._insert_edge(edge)
        return e.dest_node_index

    def _canonize_suffix(self, suffix):
        """This canonizes the suffix, walking along its suffix string until it 
        is explicit or there are no more matched nodes.
        """
        if not suffix.explicit():
            e = self.edges[suffix.source_node_index, self.string[suffix.first_char_index]]
            if e.length <= suffix.length:
                suffix.first_char_index += e.length + 1
                suffix.source_node_index = e.dest_node_index
                self._canonize_suffix(suffix)
 

    # Public methods
    def find_substring(self, substring):
        """Returns the index of substring in string or -1 if it
        is not found.
        """
        if not substring:
            return -1
        if self.case_insensitive:
            substring = substring.lower()
        curr_node = 0
        i = 0
        while i < len(substring):
            edge = self.edges.get((curr_node, substring[i]))
            if not edge:
                return -1
            ln = min(edge.length + 1, len(substring) - i)
            if substring[i:i + ln] != self.string[edge.first_char_index:edge.first_char_index + ln]:
                return -1
            i += edge.length + 1
            curr_node = edge.dest_node_index
        return edge.first_char_index - len(substring) + ln
        
    def has_substring(self, substring):
        return self.find_substring(substring) != -1

Budowa drzewa - SuffixTree("string")

In [19]:
print("Suffix - zad6")
%timeit index = SuffixTree(zad)
print("KMR - zad6")
%timeit index = kmr(zad)
print("Suffix - Romeo i Julia")
%timeit index = SuffixTree(rij)
print("KMR - Romeo i Julia")
%timeit index = kmr(rij)
print("Suffix - ustawa")
%timeit index = SuffixTree(data)
print("KMR - ustawa")
%timeit index = kmr(data)

Suffix - zad6
9.69 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMR - zad6
22.2 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Suffix - Romeo i Julia
140 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
KMR - Romeo i Julia
690 ms ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Suffix - ustawa
2.91 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
KMR - ustawa
22.2 s ± 920 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Jak widać, drzewo suffiksów buduje się szybciej niż KMR - wynika to z różnicy w złożoności obu algorytmów. Drzewo sufiksów można budować w O(n), natomiast DBF wymaga złożoności O(nlogn)

### 4. Zbadać rzeczywisty rozmiar DBF, porównać z wielkością pliku.

In [20]:
def testSize(name, txt):
    res = kmr(txt)
    print(f"File {name}: Original size - {get_deep_size(txt)}, DBF size - {get_deep_size(res)},",
          f"Ratio dbf/ori - {get_deep_size(res)/get_deep_size(txt)}")

In [21]:
testSize("Zad6", zad)
testSize("Romeo i Julia", rij)
testSize("Ustawa", data)

File Zad6: Original size - 1884, DBF size - 682112, Ratio dbf/ori - 362.0552016985138
File Romeo i Julia: Original size - 25374, DBF size - 16667580, Ratio dbf/ori - 656.8763301016788
File Ustawa: Original size - 493018, DBF size - 381244684, Ratio dbf/ori - 773.2875554239399


Jak widać DBF kosztuje bardzo dużo pamięci. Jednak musimy pamiętać że wykorzystujemy O(nlogn) pamięci, co może nie jest aż tak widoczne, przez sporą stałą.

### 5. Porównać czas wyszukiwania wzorca przy użyciu DBF z wyszukiwaniem za pomocą KMP dla różnych długości wzorca (np. jedna litera, jedno słowo, jedno zdanie).

Wykorzystamy implementację KMP:

In [22]:
# Python program for KMP Algorithm 
def KMPSearch(pat, txt, ver=False): 
    M = len(pat) 
    N = len(txt) 
    
    # create lps[] that will hold the longest prefix suffix  
    # values for pattern 
    lps = [0]*M 
    j = 0 # index for pat[] 
  
    # Preprocess the pattern (calculate lps[] array) 
    computeLPSArray(pat, M, lps) 
  
    i = 0 # index for txt[] 
    while i < N: 
        if pat[j] == txt[i]: 
            i += 1
            j += 1
  
        if j == M: 
            if ver:
                print("Found pattern at index " + str(i-j))
            j = lps[j-1] 
  
        # mismatch after j matches 
        elif i < N and pat[j] != txt[i]: 
            # Do not match lps[0..lps[j-1]] characters, 
            # they will match anyway 
            if j != 0: 
                j = lps[j-1] 
            else: 
                i += 1


def computeLPSArray(pat, M, lps): 
    len = 0 # length of the previous longest prefix suffix 
  
    lps[0] # lps[0] is always 0 
    i = 1
  
    # the loop calculates lps[i] for i = 1 to M-1 
    while i < M: 
        if pat[i]== pat[len]: 
            len += 1
            lps[i] = len
            i += 1
        else: 
            # This is tricky. Consider the example. 
            # AAACAAAA and i = 7. The idea is similar  
            # to search step. 
            if len != 0: 
                len = lps[len-1] 
  
                # Also, note that we do not increment i here 
            else: 
                lps[i] = 0
                i += 1

Wyszukanie KMPSearch(pat, txt).

In [23]:
print("zad6")
print("KMP - Letter - a")
%timeit index = KMPSearch('a', zad)
print("KMR - Letter - a")
%timeit index = findPatTxt('a', zad, False)

print("\nKMP - Word - wzorca")
%timeit index = KMPSearch('wzorca', zad)
print("KMR - Word - wzorca")
%timeit index = findPatTxt('wzorca', zad, False)

print("\nKMP - Sentence - Zaimplementować budowę słownika podstawowych składowych.")
%timeit index = KMPSearch('Zaimplementować budowę słownika podstawowych składowych.', zad)
print("KMR - Sentence - Zaimplementować budowę słownika podstawowych składowych.")
%timeit index = findPatTxt('Zaimplementować budowę słownika podstawowych składowych.', zad, False)

print("\n\nRomeo i Julia")
print("KMP - Letter - a")
%timeit index = KMPSearch('a', rij)
print("KMR - Letter - a")
%timeit index = findPatTxt('a', rij, False)

print("\nKMP - Word - piękna")
%timeit index = KMPSearch('piękna', rij)
print("KMR - Word - piękna")
%timeit index = findPatTxt('piękna', rij, False)

print("\nKMP - Sentence - Całą potomność piękności pozbawia.")
%timeit index = KMPSearch('Całą potomność piękności pozbawia.', rij)
print("KMR - Sentence - Całą potomność piękności pozbawia.")
%timeit index = findPatTxt('Całą potomność piękności pozbawia.', rij, False)

print("\n\nUstawa")
print("KMP - Letter - a")
%timeit index = KMPSearch('a', data)
print("KMR - Letter - a")
%timeit index = findPatTxt('a', data, False)

print("\nKMP - Word - Wyrób")
%timeit index = KMPSearch('Wyrób', data)
print("KMR - Word - Wyrób")
%timeit index = findPatTxt('Wyrób', data, False)

print("\nKMP - Sentence - Naprawa i odświeżanie obuwia.")
%timeit index = KMPSearch('Naprawa i odświeżanie obuwia.', zad)
print("KMR - Sentence - Naprawa i odświeżanie obuwia.")
%timeit index = findPatTxt('Naprawa i odświeżanie obuwia.', zad, False)

zad6
KMP - Letter - a
309 µs ± 34.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMR - Letter - a
25.1 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

KMP - Word - wzorca
285 µs ± 20.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMR - Word - wzorca
19.2 ms ± 620 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

KMP - Sentence - Zaimplementować budowę słownika podstawowych składowych.
261 µs ± 6.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMR - Sentence - Zaimplementować budowę słownika podstawowych składowych.
22.3 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Romeo i Julia
KMP - Letter - a
3.77 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMR - Letter - a
750 ms ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

KMP - Word - piękna
3.54 ms ± 41.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMR - Word - piękna
586 ms ± 5.43 ms per loop (mean ± std. de

Jak widać KMP jest dużo szybsze od KMR, co wynika z różnicy złożoności. KMP ma O(n), natomiast KMR ma O(nlogn).\
Pamiętajmy, że w tej wersji budujemy DBF i LPS za każdym razem.

In [24]:
print("zad6")
dbf = kmr(zad)
print("KMP - Letter - a")
%timeit index = KMPSearch('a', zad)
print("KMR - Letter - a")
%timeit index = find('a', zad, dbf, False)

print("\nKMP - Word - wzorca")
%timeit index = KMPSearch('wzorca', zad)
print("KMR - Word - wzorca")
%timeit index = find('wzorca', zad, dbf, False)

print("\nKMP - Sentence - Zaimplementować budowę słownika podstawowych składowych.")
%timeit index = KMPSearch('Zaimplementować budowę słownika podstawowych składowych.', zad)
print("KMR - Sentence - Zaimplementować budowę słownika podstawowych składowych.")
%timeit index = find('Zaimplementować budowę słownika podstawowych składowych.', zad, dbf, False)

print("\n\nRomeo i Julia")
dbf = kmr(rij)
print("KMP - Letter - a")
%timeit index = KMPSearch('a', rij)
print("KMR - Letter - a")
%timeit index = find('a', rij, dbf, False)

print("\nKMP - Word - piękna")
%timeit index = KMPSearch('piękna', rij)
print("KMR - Word - piękna")
%timeit index = find('piękna', rij, dbf, False)

print("\nKMP - Sentence - Całą potomność piękności pozbawia.")
%timeit index = KMPSearch('Całą potomność piękności pozbawia.', rij)
print("KMR - Sentence - Całą potomność piękności pozbawia.")
%timeit index = find('Całą potomność piękności pozbawia.', rij, dbf, False)

print("\n\nUstawa")
dbf = kmr(data)
print("KMP - Letter - a")
%timeit index = KMPSearch('a', data)
print("KMR - Letter - a")
%timeit index = find('a', data, dbf, False)

print("\nKMP - Word - Wyrób")
%timeit index = KMPSearch('Wyrób', data)
print("KMR - Word - Wyrób")
%timeit index = find('Wyrób', data, dbf, False)

print("\nKMP - Sentence - Naprawa i odświeżanie obuwia.")
%timeit index = KMPSearch('Naprawa i odświeżanie obuwia.', zad)
print("KMR - Sentence - Naprawa i odświeżanie obuwia.")
%timeit index = find('Naprawa i odświeżanie obuwia.', zad, dbf, False)

zad6
KMP - Letter - a
326 µs ± 105 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMR - Letter - a
81.5 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

KMP - Word - wzorca
253 µs ± 5.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMR - Word - wzorca
79.7 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

KMP - Sentence - Zaimplementować budowę słownika podstawowych składowych.
308 µs ± 125 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMR - Sentence - Zaimplementować budowę słownika podstawowych składowych.
82.4 µs ± 3.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Romeo i Julia
KMP - Letter - a
3.53 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMR - Letter - a
964 µs ± 29.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

KMP - Word - piękna
3.55 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMR - Word - piękna
958 µs ± 106 µs per loop (mean 

Tym razem policzyliśmy DBF najpierw, a później na tej podstawie wyszukiwaliśmy wzorca w tekście. Jak widać, tym razem KMR wyszukuje wzorce w tekście dużo szybciej niż KMP - oszczędzamy czas na preprocessingu LSP (LSP liczymy na wzorcu, więc nie możemy wykonać takiego preprocessingu na tekście) i tylko jednym liniowym przejściem znajdujemy wszystkie wystąpienia wzorca w teście.